# Writing high frequency trajectories when event is detected
In this tutorial we will showcase how dupin's on-line capabilites can be used to triger
HOOMD's burst writer to dump very high resolution trajectory dump but only of the event
that dupin detects. This approach carries some important consequnces for performance and
storage. The purpose of such approach is to only write to disk parts of trajctories of
interest at extremenly tight frequency, but instead of dumping the whole trajctory we
only dump part of the trajectory around the event detected by dupin.

To enable this we make a temporary buffer that stores trajectory frames in memory and
when dupin triggers this buffer is dumpeed onto the disk. On line detection requires
some CPU cycles to run every so often, which can impact performance based on order
parameters used for detection. In practice, it is often enough to track very simple
properties that are computed by the MDdriver (this case HOOMD) anyways, such as pressure
or total system energy or volume (depending if we run NVT/NVE or NPT).

Here we give one such example:


In [ ]:
import hoomd
import dupin as du
import numpy
import ruptures as rpt


cpu = hoomd.device.CPU()
simulation = hoomd.Simulation(cpu, seed=1)

# Create a simple cubic configuration of particles
N = 10  # particles per box direction
box_L = 13  # box dimension

snap = hoomd.Snapshot(cpu.communicator)
snap.configuration.box = [box_L] * 3 + [0, 0, 0]
snap.particles.N = N**3
x, y, z = numpy.meshgrid(
    *(numpy.linspace(-box_L / 2, box_L / 2, N, endpoint=False),) * 3
)
positions = numpy.array((x.ravel(), y.ravel(), z.ravel())).T
snap.particles.position[:] = positions
snap.particles.types = ['A']
snap.particles.typeid[:] = 0

simulation.create_state_from_snapshot(snap)

burst_filename = "test.gsd"

burst = hoomd.write.Burst(trigger=hoomd.trigger.Periodic(100),
                        filename=burst_filename,
                        max_burst_size=10000,
                        write_at_start=True)
simulation.operations.writers.append(burst)

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell(buffer=0.4)
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.ConstantVolume(
    filter=hoomd.filter.All(), thermostat=hoomd.md.methods.thermostats.Bussi(kT=1.5)
)
integrator.methods.append(nvt)

thermo = hoomd.md.compute.ThermodynamicQuantities(
    hoomd.filter.All()
)
simulation.operations += thermo


def custom_dupin_generator_fn(data_du):
    return{"energy":data_du}

class Online_writer(hoomd.custom.Action):
    def __init__(self, burst_writer, dupin_generator, thermo):
        self.burst_writer = burst_writer
        self.custom_generator = dupin_generator
        self.create_aggregator()
        self.thermo = thermo
        lin_regress_cost = du.detect.CostLinearFit()
        dynp = rpt.Dynp(custom_cost=lin_regress_cost)
        self.sweep_detector = du.detect.SweepDetector(dynp, max_change_points=6)

    def act(self):
        data = self.thermo["potential_energy"]
        self.signal_aggregator.accumulate(data)
        #### Also should skip detection depending on how many data points signal has, if it has 1 or two points it wont make any sense.
        # 15 is a random number here, use something better?
        # not sure if len is the correct way to figure out how many data points are there in the signal
        if len(self.signal_aggregator.signals)>15:
            if len(self.sweep_detector.fit(self.signal_aggregator.to_dataframe()))>0:
                self.burst_writer.dump()
                self.create_aggregator()

    def create_aggregator(self, dupin_generator):
        custom_generator = du.data.base.CustomGenerator(self.custom_generator)
        self.signal_aggregator = du.data.aggregate.SignalAggregator(custom_generator)


dupin_online_writer = hoomd.write.CustomWriter(action=Online_writer(burst, custom_dupin_generator_fn, thermo), trigger=1000)
simulation.operations += dupin_online_writer

simulation.run(10000)